In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import math
import time
from datetime import datetime, timedelta
import os

In [3]:
#旅遊部落客排行榜
blog_name = []
rank = 0
for i in range(7):
    page = i + 1
    r = requests.get('https://www.pixnet.net/blog/bloggers/category/16/{}'.format(page))
    r.encoding = 'utf8'
    soup = bs(r.text, 'lxml')
    for j in range(len(soup.select('#content .author'))):
        blog_name.append(soup.select('.author')[j]['href'].split('/')[-1])
        rank += 1
        if rank >= 100:
            break
blog_name

['lohas',
 'wkitty',
 'bajenny',
 'judyer101010',
 'smilejean',
 'mimg47',
 'niniblue',
 'drugs',
 'paine0602',
 'jay7134',
 'ch3126',
 'bangkokgoplay',
 'cheng0073',
 'john547',
 'j272popo',
 'fbuon2881',
 'barbieedoll',
 'viviyu',
 'crystal1983',
 'enlife',
 'venuslin0113',
 'oldfather1029',
 'windko0813',
 'l50740',
 'sweetmemory99',
 'kakakuksk',
 'pingpig',
 'evshhips',
 'a59407908',
 'nana0032458',
 'kavanachang',
 'agathachiu66',
 'cat875030',
 'banbi217',
 'difeny',
 'gugugu4411',
 'jimmraz',
 'jay0909',
 'rita11836',
 'masaharuwu',
 'happyday760626',
 'yopalit',
 'as660707',
 'amarylliss',
 'julia116',
 'maymay1231',
 'demi0130',
 'candybear0121',
 'coupine650520',
 'dreamycat0421',
 'vilily',
 'damon624',
 'tainanlohas',
 'mika0310',
 'monkey2010',
 'dong1104',
 'hrylin',
 'i88mnet',
 'bonddealerbook',
 'qpjj',
 'nanacobaby',
 'aliciaisme',
 'louis5149',
 'darrenje0322',
 'dailin114',
 'nini710',
 'aura10',
 'loveyuwa',
 'httpandkimo',
 'catty0914',
 'bulanini',
 'sinea100',


In [4]:
payload = {
    "user":"",
    "page":""
}
filename = "Pixnet-"+datetime.now().strftime('%Y%m%d')+".csv"

for bn in blog_name:
    print (bn)
    articles = []
    payload['user'] = bn
    payload['page'] = "1"
    re = requests.get('https://emma.pixnet.cc/blog/articles', params = payload)
    totalpages = math.ceil(pd.read_json(re.text)['total'][0]/pd.read_json(re.text)['per_page'][0])

    for i in range(totalpages):
        payload['page'] = i + 1
        re = requests.get('https://emma.pixnet.cc/blog/articles', params = payload)
        pd1 = pd.read_json(re.text)['articles']
        pd2 = pd.DataFrame.from_records(pd1)
        pd2['hits_total'] = 0
        pd2['pub_date'] = 'na'
        for j in range(len(pd1)):
            pd2['hits_total'].iloc[j] = pd2['hits'][j]['total']
        for k in range(len(pd2)):
            pd2['pub_date'].iloc[k] = datetime.date.fromtimestamp(int(pd2['public_at'][k])).strftime('%Y-%m-%d')

        articles.append(pd2[['category', 'category_id', 'site_category', 'site_category_id', \
                                                        'link', 'title', 'pub_date', 'hits_total']])
    Pixnet_articles = pd.concat(articles, ignore_index=True)
    if not os.path.isfile(filename):
        Pixnet_articles.to_csv(filename)
    else:
        Pixnet_articles.to_csv(filename, mode='a', header=False)

lohas


C:\Miniconda2\envs\hour_of_code\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


wkitty
bajenny
judyer101010
smilejean
mimg47
niniblue
drugs
paine0602


ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [43]:
#近期熱門部落格
r = requests.get('https://emma.pixnet.cc/mainpage/blog/categories/hot_weekly/29?per_page=100')
hot_user_name = pd.read_json(r.text)['user_name']
hot_user_name

0        benshee1005
1            whenwas
2         rosecocoya
3          cline1413
4      purpleswallow
5        Crystal1983
6          tammytwkr
7       judyer101010
8          meiko1101
9       naughtyangel
10        uniquevera
11            ch3126
12            wkitty
13       annybear920
14             lohas
15            ectomy
16              Sybs
17        mini692565
18      venuslin0113
19         chahabi77
20        eddie80128
21          zoey0830
22           bajenny
23        TWNLCC2012
24         antiwhite
25        kaikai0603
26    missrachelnina
27         kakakuksk
28      angeline5775
29             drugs
           ...      
70          twaychan
71           peko721
72        yukiko1437
73          miihuang
74         dailin114
75        fafa710117
76       eggface0507
77             anise
78      Estrella8850
79         stellahyc
80            a1630a
81        inlifefood
82        Whitesuper
83         rita11836
84            Dorcas
85          arielctl
86           